# EDA Logs QR - Gym Master

Exploratory Data Analysis (EDA) sobre los registros de ingreso por QR almacenados en Supabase.

- Extracción de datos desde Supabase
- Exploración de la distribución por hora, día, dispositivo y socio
- Detección preliminar de outliers o anomalías

**Autor:** Octavio Alvarez  
**Proyecto:** Gym Master

---

In [ ]:
# Conexión a Supabase
from supabase import create_client, Client
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

SUPABASE_URL = "https://brrxvwgjkuofcgdnmnfb.supabase.co"
SUPABASE_KEY = "<TU_SUPABASE_KEY>"
supabase: Client = create_client(SUPABASE_URL, SUPABASE_KEY)

In [ ]:
# Cargar datos de logs_qr desde Supabase
response = supabase.table('logs_qr').select('*').execute()
data = response.data

if not data:
    print("La tabla logs_qr está vacía. Ejecuta el pipeline de generación de logs.")
else:
    logs_qr_df = pd.DataFrame(data)
    print(f"{len(logs_qr_df)} registros cargados.")
    display(logs_qr_df.head())

In [ ]:
# Procesamiento si hay datos
if not data:
    print('No hay datos para analizar.')
else:
    logs_qr_df['timestamp'] = pd.to_datetime(logs_qr_df['timestamp'])
    logs_qr_df['fecha'] = pd.to_datetime(logs_qr_df['fecha'])
    logs_qr_df['dia_semana'] = logs_qr_df['timestamp'].dt.day_name()
    logs_qr_df['hora'] = logs_qr_df['hora'].astype(int)

    logs_qr_df.info()

In [ ]:
# Distribución de ingresos por hora
if not data:
    pass
else:
    plt.figure(figsize=(8,5))
    logs_qr_df['hora'].value_counts().sort_index().plot(kind='bar')
    plt.title('Distribución de ingresos por hora')
    plt.xlabel('Hora')
    plt.ylabel('Cantidad de ingresos')
    plt.show()

In [ ]:
# Distribución por día de la semana
if not data:
    pass
else:
    plt.figure(figsize=(8,5))
    logs_qr_df['dia_semana'].value_counts().reindex(
        ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']
    ).plot(kind='bar')
    plt.title('Ingresos por día de la semana')
    plt.xlabel('Día de la semana')
    plt.ylabel('Cantidad de ingresos')
    plt.show()

In [ ]:
# Uso por dispositivo
if not data:
    pass
else:
    plt.figure(figsize=(8,5))
    logs_qr_df['dispositivo'].value_counts().plot(kind='bar', color='orange')
    plt.title('Distribución por dispositivo')
    plt.xlabel('Dispositivo')
    plt.ylabel('Cantidad de ingresos')
    plt.show()

In [ ]:
# Frecuencia por socio
if not data:
    pass
else:
    plt.figure(figsize=(10,5))
    logs_qr_df['socio_id'].value_counts().plot(kind='bar', color='green')
    plt.title('Ingresos por socio')
    plt.xlabel('Socio')
    plt.ylabel('Cantidad de ingresos')
    plt.show()

In [ ]:
# Heatmap de concurrencia: Día de la semana vs Hora
if not data:
    pass
else:
    heatmap_data = logs_qr_df.groupby(['dia_semana', 'hora']).size().unstack(fill_value=0)
    heatmap_data = heatmap_data.reindex(index=['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday'])

    plt.figure(figsize=(12,6))
    sns.heatmap(heatmap_data, cmap='viridis', annot=True, fmt='d')
    plt.title('Heatmap de concurrencia por día y hora')
    plt.ylabel('Día de la semana')
    plt.xlabel('Hora del día')
    plt.show()